# Assignment 1: Wrangling and EDA
### Foundations of Machine Learning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `airbnb_NYC.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)

You end up with 0 missing values.

In [ ]:
AIRBNB = pd.read_csv('../data/airbnb_NYC.csv', encoding='latin1')
AIRBNB['Price'] = AIRBNB['Price'].str.replace(',','')
AIRBNB['Price'] = AIRBNB['Price'].astype(int)
AIRBNB['Price'].isna().sum()

2. Categorical variable: For the Minnesota police use of for data, `mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? For the remaining missing values, replace the `np.nan/None` values with the label `Missing`.

76% of `subject_injury` values are missing. "Firearm" and "Less lethal projectile" `force_type`s always have a corresponding non-nan `subject_injury` value. "Baton" has 2 nan values and (the least for a single `force_type`), and "Bodily force" has 7051 nan values (the most for a single `force_type`).

In [ ]:
MNPOLICE = pd.read_csv('../data/mn_police_use_of_force.csv')
MNPOLICE['subject_injury'].isna().sum() / MNPOLICE.shape[0] # proportion missing

In [ ]:
# print the force types that are never nan
forceTypesWhenNan = MNPOLICE[MNPOLICE['subject_injury'].isna()]['force_type'].unique()
allForceTypes = MNPOLICE['force_type'].unique()
for forceType in allForceTypes:
    if forceType not in forceTypesWhenNan: print(forceType)

In [ ]:
nas = MNPOLICE[MNPOLICE['subject_injury'].isna()]
for forceType in allForceTypes:
    print(forceType, '—', nas[nas['force_type'] == forceType].shape[0])

In [ ]:
MNPOLICE = MNPOLICE.fillna({'subject_injury': 'Missing'})
MNPOLICE['subject_injury'].unique()

3. Dummy variable: For `metabric.csv`, convert the `Overall Survival Status` variable into a dummy/binary variable, taking the value 0 if the patient is deceased and 1 if they are living.

In [ ]:
METABRIC = pd.read_csv('../data/metabric.csv')
METABRIC['Overall Survival Status'] = METABRIC['Overall Survival Status'].replace('0:LIVING', '0')
METABRIC['Overall Survival Status'] = METABRIC['Overall Survival Status'].replace('1:DECEASED', '1')
METABRIC['Overall Survival Status'] = METABRIC['Overall Survival Status'].astype(int)
METABRIC['Overall Survival Status'].unique()

4. Missing values: For `airbnb_NYC.csv`, determine how many missing values of `Review Scores Rating` there are. Create a new variable, in which you impute the median score for non-missing observations to the missing ones. Why might this bias or otherwise negatively impact your results?

There are 8323 missing values of `Review Scores Rating`. This might impact my results because it might under or overrepresent the review scores of certain places. Imputing missing values as the median means that many properties are going to be regarded as just average. However, in acutuality, a reasonable amount of those are probably better than average and a reasonable amount are worse than average. In other words, imputing the median could lead to bias towards the middle.

In [ ]:
AIRBNB['Review Scores Rating'].isna().sum()

In [ ]:
AIRBNB2 = AIRBNB.copy()
AIRBNB2 = AIRBNB2.fillna({'Review Scores Rating': AIRBNB2['Review Scores Rating'].median()})
AIRBNB2['Review Scores Rating'].isna().sum()

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work. What does work?
2. Drop any columns that do not contain data.
3. What is an observation? Carefully justify your answer, and explain how it affects your choices in cleaning and analyzing the data.
4. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
5. Clean the Age variable and make a histogram of the ages of the victims.
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Thoughts?

In [ ]:
#1
# read_execel works
SHARKS = pd.read_excel('../data/GSAF5.xls')

#2
SHARKS.dropna(axis='columns', how='all', inplace=True)
SHARKS = SHARKS.drop(columns=['Unnamed: 21', 'Unnamed: 22'])

SHARKS.head()

3. Each observation is one shark attack. It includes info about the location, the type of shark, the activity of the person attacked, etc. This affects my choices in cleaning the data because it means that I shouldn't drop too many rows (eg. for having missing values). I don't want to lose too many shark attacks.

In [ ]:
# 4

# code to find weird things
SHARKS['Year'].unique() # weird years are 77, 5, and 0
yearIsZero = SHARKS[SHARKS['Year'] == 0]
yearIsZero = yearIsZero[yearIsZero['Date'].str.contains('B')]
yearIsZero = yearIsZero[yearIsZero['Date'].str.contains('C')]
yearIsZero

The range of values is from 2026 all the way back to 5. It also includes some incidents from BC years.

In [ ]:
# code to fix weird things

# 77 is acceptable, it is for an attack in 77 AD
# 5 is also acceptable, it is for an attack in 5 AD

# 0 is for a bunch of things :D no dates, ranges, questions, etc.
# for simplicity, I change all these into nan values
SHARKS['Year'] = SHARKS['Year'].replace(0, np.nan)
SHARKS['Year'].unique()

In [ ]:
# filter rows to only include years > 1940
SHARKS2 = SHARKS.copy()
SHARKS2 = SHARKS[SHARKS['Year'] >= 1940]

# plot to see trend over time
counts = SHARKS2['Year'].value_counts().to_frame()
sns.catplot(counts, kind='bar', x='Year', y='count', aspect=3)
plt.xticks(rotation=90)
plt.suptitle("Sharks Attacks Over Time")
plt.show()


Sharks attacks have generally increased since 1940.

In [ ]:
# 5

# code to find and fix weird things
# anything that is weird becomes nan for simplicity
agesInData = SHARKS2['Age'].unique()
weirdAges = []
for age in agesInData:
    if not str(age).isdigit():
        weirdAges.append(age)

SHARKS3 = SHARKS2.copy()
SHARKS3['Age'] = SHARKS3['Age'].replace(weirdAges, np.nan)
SHARKS3['Age'] = SHARKS3['Age'].astype(float)
SHARKS3['Age'].unique()

In [ ]:
# plot as histogram

sns.catplot(SHARKS3.sort_values('Age'), x='Age', kind='count', aspect=3)
plt.xticks(rotation=90)
plt.title('histogram of ages')
plt.show()

In [ ]:
# 6

# code to find weird things
SHARKS4 = SHARKS3.copy()
SHARKS4['Type'].unique()


In [ ]:
# code to fix weird things
toUnknown = ['Questionable', 'Watercraft', 'Sea Disaster', np.nan, '?', 'Unconfirmed', 'Unverified', 'Invalid', 'Under investigation', 'Boat']
SHARKS4['Type'] = SHARKS4['Type'].replace(toUnknown, 'Unknown')
SHARKS4['Type'] = SHARKS4['Type'].replace('unprovoked', 'Unprovoked')
SHARKS4['Type'] = SHARKS4['Type'].replace(' Provoked', 'Provoked')
SHARKS4['Type'].unique()

In [ ]:
# find proportion unprovoked
len(SHARKS4[SHARKS4['Type'] == 'Unprovoked']) / len(SHARKS4['Type'])

About 74% of the shark attacks were unprovoked.

In [ ]:
# 7 

# code to find weird things
SHARKS5 = SHARKS4.copy()
SHARKS5['Fatal Y/N'] = SHARKS5['Fatal Y/N'].str.strip()
SHARKS5['Fatal Y/N'].unique()

In [ ]:
# code to fix weird things
toY = ['F', 'Y x 2']
toN = ['n', 'Nq']
toUnknown2 = ['M', np.nan, 'UNKNOWN']
SHARKS5['Fatal Y/N'] = SHARKS5['Fatal Y/N'].replace(toY, 'Y')
SHARKS5['Fatal Y/N'] = SHARKS5['Fatal Y/N'].replace(toN, 'N')
SHARKS5['Fatal Y/N'] = SHARKS5['Fatal Y/N'].replace(toUnknown2, 'Unknown')
SHARKS5['Fatal Y/N'].unique()

In [ ]:
# 8

types = SHARKS5['Type'].unique()

fatalityByType = pd.DataFrame({
    'type': types,
    'percentFatal': [0.0, 0.0, 0.0]
})
fatalityByType = fatalityByType.set_index('type')

for type in types:
    fatalColumn = SHARKS5[SHARKS5['Type'] == type]['Fatal Y/N']
    percentFatal = ( len(fatalColumn[fatalColumn == 'Y']) / len(fatalColumn) ) * 100
    fatalityByType.loc[type, 'percentFatal'] = percentFatal
    
fatalityByType

Unprovoked attacks are more likely to be fatal than provoked attacks. This is the opposite of what I would have suspected. Maybe this is because some of the unknown attacks should have been marked as provoked.

**Q3.** Open the "tidy_data.pdf" document available in `https://github.com/ds4e/wrangling`, which is a paper called *Tidy Data* by Hadley Wickham.

  1. Read the abstract. What is this paper about?

  This paper is about the concept of "tidy data" which is data where each variable is a column, each observation is a row, and each type of observational unit is a table. Tidy data makes it easier to work with data.

  2. Read the introduction. What is the "tidy data standard" intended to accomplish?

  The "tidy data standard" is intended to facilitate initial data exploration and analysis and to simplify th development of data analysis tools that work together. It aims to make data analysis easier and draw focus away from uninteresting logistics.

  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."

  The first sentence ("Like families...") means that all nice, tidy datasets are standardized and work together. It also means that messy datasets are all varied and structured in different ways. Tidy datasets give structure to messy datasets.

  The second sentence ("For a given dataset...") means that what should be a variable and what should be an observation can vary by dataset. Wickham gives an example: in one dataset, there could be a `height` variable and a `width` variable; but in another dataset, there could be a `dimension` variable with possible values of `height` and `width`.

  4. Read Section 2.2. How does Wickham define values, variables, and observations?

  Values are defined as numbers or strings (usually) that each belong to a variable and an observation. Variables are groupings of values that all measure the same attribute across units, for example, height. And observations are groupings of values that all measure the same unit across attributes, for example, people.

  5. How is "Tidy Data" defined in section 2.3?

  "Tidy Data" is defined by three characteristics: 1) each variable must form a column, 2) each observation must form a row, and 3) each type of observational unit must form a table.

  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?

  The five most common problems with messy datasets are 1) columns are values instead of variables, 2) multiple variables are in one column, 3) variables are in both rows and columns, 4) multiple types of observational units are in the same table, and 5) a single observational unit is in multiple tables.

  The data in Table 4 are messy because the values for the variable `income` are in the columns.

  "Melting" a dataset is taking the columns that are not already variables and turning them into variables. You do this by making a new variable called `column` that contains repeated column headings and by making a new variable `value` with the concatenated data values from the previously separate columns.

  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?

  Table 11 is messy because variables are stored in both columns and rows. For example, the variables `id`, `year`, and `month` are in columns but the variables `tmin` and `tmax` are in rows. Table 12 is tidy and molten because it follows the aforementioned three precepts of Tidy Data. The data in Table 12 becomes molten as the variable names are put into an `element` column, and it becomes tidy as the row variables `tmin` and `tmax` are turned into columns. 

**Q4.** This question looks at financial transfers from international actors to American universities. In particular, from which countries and giftors are the gifts coming from, and to which institutions are they going? 

For this question, `.groupby([vars]).count()` and `.groupby([vars]).sum()` will be especially useful to tally the number of occurrences and sum the values of those occurrences.

1. Load the `ForeignGifts_edu.csv` dataset.
2. For `Foreign Gift Amount`, create a histogram and describe the variable. Describe your findings.
3. For `Gift Type`, create a histogram or value counts table. What proportion of the gifts are contracts, real estate, and monetary gifts?
4. What are the top 15 countries in terms of the number of gifts? What are the top 15 countries in terms of the amount given?
5. What are the top 15 institutions in terms of the total amount of money they receive? Make a histogram of the total amount received by all institutions. 
6. Which giftors provide the most money, in total? 

In [ ]:
# 1
GIFTS = pd.read_csv('../data/ForeignGifts_edu.csv')
GIFTS.head()

In [ ]:
# 2
sns.histplot(GIFTS['Foreign Gift Amount'])
plt.show()

In [ ]:
sns.histplot(np.log(GIFTS['Foreign Gift Amount']))
plt.show()

In [ ]:
print(GIFTS['Foreign Gift Amount'].min())
print(GIFTS['Foreign Gift Amount'].max())

The histogram has a long tail, which means that most gifts are not too much money and that few gifts are actually reaching extraoridinary amounts. The `Foreign Gift Amount` variable has a wide range from -537770 all the way to 99999999.

In [ ]:
# 3
typeCounts = GIFTS['Gift Type'].value_counts().to_frame()
totalCount = len(GIFTS['Gift Type'])
typeCounts['proportion'] = typeCounts['count'] / totalCount
typeCounts

About 61% of gifts are contracts, about 39% of gifts are monetary, and about 0.03% of gifts are real estate.

In [ ]:
# 4
giftsByCountry = GIFTS.groupby(['Country of Giftor'])['ID'].count().to_frame()
giftsByCountry.columns = ['number of gifts']
giftsByCountry.sort_values('number of gifts', ascending=False).head(15)

In [ ]:
giftAmtsByCountry = GIFTS.groupby(['Country of Giftor'])['Foreign Gift Amount'].sum().to_frame()
giftAmtsByCountry.columns = ['amount of gifts']
giftAmtsByCountry.sort_values('amount of gifts', ascending=False).head(15)

In [ ]:
# 5 
giftsReceivedByInstitutions = GIFTS.groupby(['Institution Name'])['Foreign Gift Amount'].sum().to_frame()
giftsReceivedByInstitutions.columns = ['amount received']
giftsReceivedByInstitutions.sort_values('amount received', ascending=False).head(15)

In [ ]:
sns.histplot(giftsReceivedByInstitutions)
plt.show()

In [ ]:
# 6
giftsGivenByGiftor = GIFTS.groupby(['Giftor Name'])['Foreign Gift Amount'].sum().to_frame()
giftsGivenByGiftor.columns = ['amount given']
giftsGivenByGiftor.sort_values('amount given', ascending=False).head(15)

**Q5.** This question provides some practice doing exploratory data analysis and visualization.

We'll use the `college_completion.csv` dataset from the US Department of Education. The "relevant" variables for this question are:
  - `level` - Level of institution (4-year, 2-year)
  - `aid_value` - The average amount of student aid going to undergraduate recipients
  - `control` - Public, Private not-for-profit, Private for-profit
  - `grad_100_value` - percentage of first-time, full-time, degree-seeking undergraduates who complete a degree or certificate program within 100 percent of expected time (bachelor's-seeking group at 4-year institutions)

1. Load the `college_completion.csv` data with Pandas.
2. How many observations and variables are in the data? Use `.head()` to examine the first few rows of data.
3. Cross tabulate `control` and `level`. Describe the patterns you see in words.
4. For `grad_100_value`, create a kernel density plot and describe table. Now condition on `control`, and produce a kernel density plot and describe tables for each type of institutional control. Which type of institution appear to have the most favorable graduation rates?
5. Make a scatterplot of `grad_100_value` by `aid_value`, and compute the covariance and correlation between the two variables. Describe what you see. Now make the same plot and statistics, but conditioning on `control`. Describe what you see. For which kinds of institutions does aid seem to vary positively with graduation rates?

In [ ]:
# 1
COLLEGE = pd.read_csv('../data/college_completion.csv')

In [ ]:
# 2
COLLEGE.head()

In [ ]:
COLLEGE.columns

In [ ]:
COLLEGE.shape

There are 63 columns in the dataset and thus 63 variables. However, `index` and `unitid` seem to serve more as identifiers rather than actual measured variables. So, there are really 61 variables in the dataset. And, there are 3798 rows in the dataset and thus 3798 observations.

In [ ]:
COLLEGE['control'].unique()

In [ ]:
# 3
pd.crosstab(COLLEGE['control'], COLLEGE['level'])

Most 2-year colleges are public, but most 4-year colleges are private non-profit. An overwhelming majority of private non-profit schools are 4-year. Private for-profit schools are somewhat evenly split between 2-year and 4-year programs, while public colleges tend towards 2-year programs.

In [ ]:
# 4
sns.kdeplot(COLLEGE['grad_100_value'])
plt.show()

In [ ]:
COLLEGE['grad_100_value'].describe().to_frame()

In [ ]:
sns.displot(COLLEGE, kind='kde', x='grad_100_value', hue='control')
plt.title('100% time completion rate density by control of college')
plt.xlabel('100% time completion rate')
plt.show()

In [ ]:
for control in COLLEGE['control'].unique():
    print(control)
    print(COLLEGE[COLLEGE['control'] == control]['grad_100_value'].describe())
    print()

Private non-profit schools seem to have the most favorable graduation rate. Most public schools have a lower graduation rate, as seen by the high peak on the left in the kde plot and the mean of about 17. Continuing to look at the means, we see that private non-protfit schools surpass private for-profit schools with a mean of 41 over a mean of 29. Also, in the kde plot, we see that the private non-profit schools have higher density on the right (than private for-profit schools) which further indicates that provie non-profit schools have the most favorable graduation rate.

In [ ]:
# 5
sns.scatterplot(COLLEGE, x='aid_value', y='grad_100_value')
plt.title('graduation rate by aid received')
plt.xlabel('aid received')
plt.ylabel('graduation rate')
plt.show()

In [ ]:
COLLEGE2 = COLLEGE[['grad_100_value', 'aid_value']]
COLLEGE2.cov()

In [ ]:
COLLEGE2.corr()

The graduation rate and aid amount seem to only somewhat related to one another. In the scatter plot, I see that the graduation rate tends upwards as the aid amount goes up, and this is affirmed by the postive covariance and correlation values.

In [ ]:
sns.scatterplot(COLLEGE, x='aid_value', y='grad_100_value', hue='control')
plt.title('graduation rate by aid received')
plt.xlabel('aid received')
plt.ylabel('graduation rate')
plt.show()

In [ ]:
for control in COLLEGE['control'].unique():
    COLLEGE3 = COLLEGE[COLLEGE['control'] == control][['grad_100_value', 'aid_value']]
    print(control)
    print(COLLEGE3.cov())
    print()


In [ ]:
for control in COLLEGE['control'].unique():
    COLLEGE3 = COLLEGE[COLLEGE['control'] == control][['grad_100_value', 'aid_value']]
    print(control)
    print(COLLEGE3.corr())
    print()

For private non-profit colleges, graduation rate and aid amount seem to be the most strongly related. This is seen in the scatter plot with most of the points that represent higher graduation rates and higher aid amounts being from private non-profit colleges. The covariance and correlation values for private non-profit schools are also higher than that of public schools and private for-profit schools.

**Q6.** In class, we talked about how to compute the sample mean of a variable $X$,
$$
m(X) = \dfrac{1}{N} \sum_{i=1}^N x_i
$$
and sample covariance of two variables $X$ and $Y$,
$$
\text{cov}(X,Y) = \dfrac{1}{N} \sum_{i=1}^N (x_i - m(X))(y_i - m(Y))).
$$
Recall, the sample variance of $X$ is
$$
s^2 = \dfrac{1}{N} \sum_{i=1}^N (x_i - m(X))^2.
$$
It can be very helpful to understand some basic properties of these statistics. If you want to write your calculations on a piece of paper, take a photo, and upload that to your GitHub repo, that's probably easiest.

We're going to look at **linear transformations** of $X$, $Y = a + bX$. So we take each value of $X$, $x_i$, and transform it as $y_i = a + b x_i$. 

1. Show that $m(a + bX) = a+b \times m(X)$.
2. Show that $ \text{cov}(X,X) = s^2$.
3. Show that $\text{cov}(X,a+bY) = b \times \text{cov}(X,Y)$
4. Show that $\text{cov}(a+bX,a+bY) = b^2 \text{cov}(X,Y) $. Notice, this also means that $\text{cov}(bX, bX) = b^2 s^2$.
5. Suppose $b>0$ and let the median of $X$ be $\text{med}(X)$. Is it true that the median of $a+bX$ is equal to $a + b \times \text{med}(X)$? Is the IQR of $a + bX$ equal to $a + b \times \text{IQR}(X)$?
6. Show by example that the means of $X^2$ and $\sqrt{X}$ are generally not $(m(X))^2$ and $\sqrt{m(X)}$. So, the results we derived above really depend on the linearity of the transformation $Y = a + bX$, and transformations like $Y = X^2$ or $Y = \sqrt{X}$ will not behave in a similar way.

My answers to these problems are in the same GitHub repo that this notebook is in — also linked [here](https://github.com/syd-nguyen/ds3021-assignments/blob/main/assignments/01_Q6.pdf).

**Q7.** This question provides some practice doing exploratory data analysis and visualization.

We'll use the `ames_prices.csv` dataset. The "relevant" variables for this question are:
  - `price` - Sale price value of the house
  - `Bldg.Type` - Building type of the house (single family home, end-of-unit townhome, duplex, interior townhome, two-family conversion)

1. Load the `ames_prices.csv` data with Pandas.
2. Make a kernel density plot of price and compute a describe table. Now, make a kernel density plot of price conditional on building type, and use `.groupby()` to make a describe type for each type of building. Which building types are the most expensive, on average? Which have the highest variance in transaction prices?
3. Make an ECDF plot of price, and compute the sample minimum, .25 quantile, median, .75 quantile, and sample maximum (i.e. a 5-number summary).
4. Make a boxplot of price. Are there outliers? Make a boxplot of price conditional on building type. What patterns do you see?
5. Make a dummy variable indicating that an observation is an outlier.
6. Winsorize the price variable, and compute a new kernel density plot and describe table. How do the results change?

In [ ]:
# 1
AMES = pd.read_csv('../data/ames_prices.csv')
AMES.head()

In [ ]:
# 2
sns.kdeplot(AMES['price'])
plt.show()

In [ ]:
AMES['price'].describe().to_frame()

In [ ]:
sns.displot(AMES, kind='kde', x='price', hue='Bldg.Type', common_norm=False)
plt.show()

In [ ]:
AMES.groupby(['Bldg.Type'])['price'].describe()

Townhome end units (`TwnhsE`) are the most expensive on average. 1-family homes have the highest variance (and standard deviation) in price.

In [ ]:
# 3
sns.ecdfplot(AMES['price'])
plt.show()

In [ ]:
fiveNumSum = pd.DataFrame({
    'var': ['min', '25%', 'median', '75%', 'max'],
    'value': [
        AMES['price'].min(),
        np.quantile(AMES['price'], 0.25),
        np.quantile(AMES['price'], 0.5),
        np.quantile(AMES['price'], 0.75),
        AMES['price'].max()
    ]
})
fiveNumSum = fiveNumSum.set_index('var')
fiveNumSum

In [ ]:
# 4
sns.catplot(AMES, kind='box', x='price', aspect=2)
plt.show()

Yes. There are quite a few outliers on the upper end of price.

In [ ]:
sns.catplot(AMES, kind='box', x='price', hue='Bldg.Type', aspect=2)
plt.show()

1-family homes have the most outliers on the higher end of price, even though their mean price is within the mean prices of other building types. They also have a wide range of prices. Townhouse end units have a higher mean price than townhouse inside units. The spread of duplex and 2-family conversion homes seems to be smaller than that of other building types.

In [ ]:
# 5
q25 = np.quantile(AMES['price'], 0.25)
q75 = np.quantile(AMES['price'], 0.75)
isLowerOutlier = (AMES['price'] < q25).astype(int)
isUpperOutlier = (AMES['price'] > q75).astype(int)
isOutlier = isLowerOutlier + isUpperOutlier

In [ ]:
# 6
# 6. Winsorize the price variable, and compute a new kernel density plot and describe table. How do the results change?
iqr = q75 - q25
uw = q75 + 1.5 * iqr
lw = q25 - 1.5 * iqr
winsorize = (
    isUpperOutlier * uw +
    isLowerOutlier * lw +
    (1-isOutlier) * AMES['price']
)

In [ ]:
sns.scatterplot(x=AMES['price'], y=winsorize)

In [ ]:
sns.kdeplot(x=winsorize)
plt.show()

In [ ]:
winsorize.describe()

In [ ]:
AMES['price'].describe()

There are now three peaks in the kdeplot because of the winsorizing. I think this is because of all the upper outliers and all the lower outliers being set to new values. Those new values are now very dense. Also, the mean has decreased, while the standard deviation has increased. The minimum and the maximum have decreased. The quartiles are the same.